## Tratamento do arquivo 'agenda.csv'
Neste caderno será apresentada a exploração inicial e tratamento dos dados do arquivo 'agenda.csv'

### Importação das bibliotecas
 
A seguir uma breve apresentação de cada biblioteca que será utilizada:

- A biblioteca [NumPy](https://numpy.org/) é fundamental para qualquer tipo de computação científica em Python
- A biblioteca [pandas](https://pandas.pydata.org/) é a nossa ferramenta pricipal para análise e manipulação de dados

In [15]:
import numpy as np
import pandas as pd

### Leitura e tratamento inicial dos dados de entrada

Através da biblioteca `pandas` é realizada a leitura do arquivo '.csv' utilizando a função `read_csv()`, onde cada parâmetro tem o seguinte significado:

- `parse_dates=['Data']`: Fazer a importação da coluna `'Data'` como tipo `data hora`
- `encoding='iso-8859-1'`: Tipo de `encoding` do arquivo sendo lido
- `quotechar='"'`: O caracter usado para denotar o inicio e fim de um item entre aspas
- `delimiter='|'`: delimitador utilizado na escrita do arquivo

In [16]:
df = pd.read_csv('desafio-base1/agenda.csv', parse_dates=['Data'], encoding='iso-8859-1', quotechar='"', delimiter='|')

(1230, 9)

### Visualização dos dados

Nas próximas duas células é realizada a visualização inicial dos dados.

Usando o método `head()` do `pandas` com um argumento `5` nele é possível visualizar os primeiros `5` registros do Dataframe.
    
O `.T` significa `Transposição`, desta forma as linhas serão visualizadas como colunas e vice-versa.

In [17]:
df.head(5).T

,0,1,2
Código,91,100,564
Data,2020-09-10 00:00:00,1997-06-01 00:00:00,1999-09-10 00:00:00
CodConvenio,9.0,13.0,2.0
HoraConsulta,14:13:35,NaN,18:31:24
Tipo,1ª vez,Consulta,Retorno
Valor,107.0,123.0,212.0
HoraAtendimento,19:00:49,NaN,22:31:27
HoraFim,22:15:06,NaN,23:11:29
Usuário,José,Teste 1,Teste 1


O método `info()` do `pandas` apresenta um resumo dos dados no Dataframe, uma informação interessante é o tipo de dado de cada recurso.

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1230 entries, 0 to 1229
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Código           1230 non-null   int64         
 1   Data             1230 non-null   datetime64[ns]
 2   CodConvenio      986 non-null    float64       
 3   HoraConsulta     984 non-null    object        
 4   Tipo             995 non-null    object        
 5   Valor            977 non-null    float64       
 6   HoraAtendimento  984 non-null    object        
 7   HoraFim          984 non-null    object        
 8   Usuário          1230 non-null   object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 86.6+ KB


## Limpeza e tratamento dos dados

A seguir será realizada a limpeza e tratamento dos dados.

### Valores ausentes, valores equivalentes e tratamento inicial

Quando utilizamos o método `info()` para ver o resumo dos dados, foi possível ver que muitas colunas tinham muitos dados ausentes, entrentanto, na documentação da iClinic é possível ver que os campos `patient_name`, `physician_id` e `date` são obrigatórios e no Dataset anterior as colunas `Usuário`, `Código` e `Data` são seus equivalentes, ainda, estas são as únicas colunas que não tem valores nulos nos seus registros, logo, a priori, não será necessário um maior tratamento para cumprir as condições obrigatórias.

O tratamento de algumas colunas será realizado em seguida, mas quando o tratamento necessário for apenas na mudança nos nomes entre os recursos equivalentes, este será realizada no final, já que não é relevante para o tratamento inicial.

Alguns dos recursos precisarão de uma atenção ou tratamento mais detalhada, são estes: "Tipo" e "Valor".

### Tratamento dos recursos 'HoraConsulta', 'HoraAtendimento' e 'HoraFim'
 
Nestes recursos será realizada apenas a formatação das horas como solicitado no padrão iClinic, ainda, o author não conseguiu perceber ou encontrar o valor para a 'data de entrada', pelo que será deixado apenas o tempo na coluna do recurso 'HoraConsulta', como mostrado a seguir:


In [19]:
df['HoraConsulta'] = pd.to_datetime(df['HoraConsulta'], format='%H:%M:%S').dt.time
df['HoraAtendimento'] = pd.to_datetime(df['HoraAtendimento'], format='%H:%M:%S').dt.time
df['HoraFim'] = pd.to_datetime(df['HoraFim'], format='%H:%M:%S').dt.time

### Tratamento do recurso 'patient_name'

Este recurso precisa de informação externa para ser tratado, pelo que, após a importação do arquivo "patient.csv", onde está pode ser encontrada dentre outras informações, o nome do paciente, será utilizado o índice 'patient_id' para localizar o nome de cada usuário.

In [20]:
df_patient = pd.read_csv('desafio-base1-output/patient.csv', usecols = ["name"])

Repare que estamos utilizando o arquivo com os dados tratados anteriormente.
Neste caso, como o 'patient_id' inicia com 0 e está ordenada é fácil de encontrar seu nome correspondente.

In [21]:
index = df['Código'].values
df['patient_name'] = df_patient['name'][index].values

### Tratamento do recurso 'physician_id'

Será criado um novo recurso com o nome 'physician_id', como a seguir:

In [22]:
df['physician_id'] = pd.factorize(df['Usuário'])[0].astype(int)

O método `factorize(df['Usuário'])` realiza o mapeamento dos valores do recurso, dando um número para cada usuário em `df['Usuário']` de forma sequencial.

### Exportação dos nomes dos médicos

Aqui o autor achou interessante criar uma lista relacionando o nome do médico e seu 'physician_id' do médico para futuro tratamento, a seguir será exportado um arquivo `.csv` com os nomes dos médicos de forma ordenada.

In [33]:
uniques = pd.factorize(df['Usuário'])[1]
pd_uniques = pd.DataFrame(data={"physician_names": uniques})
pd_uniques.to_csv('desafio-base1-output/physician_names.csv',index=False, encoding='utf-8')

### Tratando o recurso 'CodConvenio'
 
Não foi possível tratar este recurso pela falta dos nomes dos convênios, mas é importante enfatizar que o tratamento deste rescurso seria semelhante ao realizado no caderno anterior na seção `Tratando o recurso 'EstadoCivil'`.

### Tratamento do recurso 'eventprocedure_pack'

Será criado um novo recurso com o nome 'eventprocedure_pack', possuira uma string em formato `json`, pelo que será utilizado o método `to_json` nas colunas 'Tipo' e 'Valor'.

Neste caso foi utilizado parâmetro `force_ascii = False` para visualizar a escrita de forma correta em `utf-8`.

Além disso, como não foi possível obter infromação sobre o recurso 'quantity', não será colocado nada no lugar.

In [9]:
df['eventprocedure_pack'] = ('json::['+df[['Tipo','Valor']].apply(lambda x: x.to_json(force_ascii =  False),axis=1)+']')

### Mudança de nome das colunas para o padrão iClinic

Em seguida será realizada a mudança no nome dos recursos no Dataset anterior para os nomes equivalente ao padrão iClinic:

In [10]:
df = df.rename(
    columns = {
        "Código": "patient_id",
        "Data": "date",
        "HoraConsulta": "arrival_time",
        "HoraAtendimento": "start_time",
        "HoraFim": "end_time",
        "CodConvenio": "healthinsurance_name",
        "Tipo": "name",
        "Valor": "value",
    }
)

### Adição de recursos ausentes

In [11]:
df['status'] = 'cp'
df['patient_home_phone'] = np.nan
df['patient_mobile_phone'] = np.nan
df['description'] = np.nan
df['all_day'] = np.nan
df['cancel_reason'] = np.nan  
df['patient_email'] = np.nan  
df['event_blocked_scheduling'] = np.nan
df['quantity'] = np.nan  

### Remoção de recursos repetidos ou não necessários

A seguir, serão removidos os recursos repetidos ou que não são mais necessários:

In [12]:
df = df.loc[:, ["patient_id","patient_name","physician_id","date","status","patient_home_phone","patient_mobile_phone","arrival_time","start_time","end_time","description","all_day","cancel_reason","patient_email","event_blocked_scheduling","healthinsurance_name","eventprocedure_pack"]]

## Exportação do arquivo de saída

Como solicitado no desafio, o arquivo de saída será gerado com o conjunto de caracteres `UTF-8`:

In [13]:
df.to_csv('desafio-base1-output/event_scheduling.csv',index=False, encoding='utf-8')